source:

https://www.33rdsquare.com/text-analysis-topic-modeling-with-bert/?utm_source=chatgpt.com

https://link.springer.com/article/10.1007/s41870-023-01268-w?utm_source=chatgpt.com



In [11]:
import pandas as pd
from transformers import BertTokenizer, BertModel
import torch
import numpy as np
from sklearn.cluster import KMeans
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.datasets import fetch_20newsgroups
from gensim.models.coherencemodel import CoherenceModel
from gensim.corpora.dictionary import Dictionary

import nltk
from collections import Counter
nltk.download('punkt')
from nltk.tokenize import word_tokenize
import string

import nltk
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer
from transformers import BertTokenizer, BertModel
import torch
from gensim.models.coherencemodel import CoherenceModel

# Download required NLTK resources
nltk.download('stopwords')
nltk.download('punkt')
nltk.download('wordnet')

[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sajjadislam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to
[nltk_data]     /Users/sajjadislam/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to
[nltk_data]     /Users/sajjadislam/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to
[nltk_data]     /Users/sajjadislam/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


True

In [35]:
# Step 1: Load the dataset
newsgroups = fetch_20newsgroups(subset='all', remove=('headers', 'footers', 'quotes'))

# Step 2: Create a DataFrame for better handling
df = pd.DataFrame({
    'Text': newsgroups.data,
    'Category': [newsgroups.target_names[label] for label in newsgroups.target]
})

In [36]:
print(df.shape)

# Step 1: Count words in the 'Text' column
df['WordCount'] = df['Text'].apply(lambda x: len(x.split()))

# Step 2: Calculate the thresholds for the bottom and top 10%
lower_threshold = df['WordCount'].quantile(0.1)
upper_threshold = df['WordCount'].quantile(0.9)

# Step 3: Filter the DataFrame to exclude bottom 10% and top 10%
df = df[(df['WordCount'] > lower_threshold) & (df['WordCount'] < upper_threshold)]

# Step 2: Calculate statistics
median_word_count = df['WordCount'].median()
min_word_count = df['WordCount'].min()
max_word_count = df['WordCount'].max()

# Display the results
print("Median Word Count:", median_word_count)
print("Minimum Word Count:", min_word_count)
print("Maximum Word Count:", max_word_count)

# Display the filtered DataFrame
print(df.shape)

(18846, 2)
Median Word Count: 83.0
Minimum Word Count: 19
Maximum Word Count: 330
(15021, 3)


In [37]:
df.dropna(subset=['Text'], inplace=True)
# Randomly sample 500 rows (use the actual dataset size if it's smaller than 500)
df = df.sample(n=min(500, len(df)), random_state=42)

# Rename 'title' column to 'Comment'
df.rename(columns={'Text': 'Comment'}, inplace=True)

df.shape

(500, 3)

In [38]:
#import torch

# Ensure GPU availability
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Using device: {device}")

Using device: cpu


In [44]:
# Load pre-trained BERT model and tokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Function to encode comments into embeddings
def encode_comments(texts):
    encoded_input = tokenizer(texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state.mean(dim=1)
    return embeddings

# Encode comments
embeddings = encode_comments(df['Comment'].tolist())


In [42]:
# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased')

# Set of English stopwords and initialize lemmatizer
stop_words = set(stopwords.words('english'))
# Include all digits and symbols
custom_stop_words = set(string.digits + string.punctuation) 
additional_stop_words = ['the', 'and', 'was', 'were', 'with', 'a']

stop_words.update(custom_stop_words)
stop_words.update(additional_stop_words)

lemmatizer = WordNetLemmatizer()

def encode_comments(texts):
    # Function to remove stopwords
    def remove_stopwords(text):
        word_tokens = word_tokenize(text)
        filtered_text = [word for word in word_tokens if word.lower() not in stop_words]
        return " ".join(filtered_text)

    # Function to remove stopwords and lemmatize
    import string

    def preprocess_text(text):
        # Remove punctuation from text
        translator = str.maketrans('', '', string.punctuation)
        text = text.translate(translator).lower()  # Lowercasing the text here

        word_tokens = word_tokenize(text)
        filtered_and_lemmatized_text = [
            lemmatizer.lemmatize(word)
            for word in word_tokens
            if word not in stop_words
        ]
        return " ".join(filtered_and_lemmatized_text)

    # Remove stopwords from each comment
    cleaned_texts = [preprocess_text(text) for text in texts]

    # Tokenization and encoding
    encoded_input = tokenizer(cleaned_texts, padding=True, truncation=True, return_tensors='pt')
    with torch.no_grad():
        model_output = model(**encoded_input)
    embeddings = model_output.last_hidden_state.mean(dim=1)
    return embeddings

# Encode comments
embeddings = encode_comments(df['Comment'].tolist())

In [39]:
# Initialize BERT tokenizer and model
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
model = BertModel.from_pretrained('bert-base-uncased').to(device)

# Stop words and lemmatizer
stop_words = set(stopwords.words('english')).union(set(string.digits + string.punctuation))
lemmatizer = WordNetLemmatizer()

def preprocess_text(text):
    translator = str.maketrans('', '', string.punctuation)
    text = text.translate(translator).lower()
    word_tokens = word_tokenize(text)
    filtered_text = [
        lemmatizer.lemmatize(word) for word in word_tokens if word not in stop_words
    ]
    return " ".join(filtered_text)

# Step 3: Preprocess comments
df['Cleaned_Comment'] = df['Comment'].apply(preprocess_text)

def encode_comments_batch(texts, batch_size=16):
    embeddings = []
    for i in range(0, len(texts), batch_size):
        batch_texts = texts[i:i + batch_size]
        encoded_input = tokenizer(batch_texts, padding=True, truncation=True, return_tensors='pt').to(device)
        with torch.no_grad():
            model_output = model(**encoded_input)
        batch_embeddings = model_output.last_hidden_state.mean(dim=1).cpu()
        embeddings.append(batch_embeddings)
    return torch.cat(embeddings, dim=0)

# Encode comments in batches
embeddings = encode_comments_batch(df['Cleaned_Comment'].tolist())

In [45]:
# Perform clustering on embeddings (example: K-Means)
num_topics = 10  # Set the number of topics
kmeans = KMeans(n_clusters=num_topics, random_state=0).fit(embeddings.numpy())
df['topic'] = kmeans.labels_

# For Coherence Score, transform BERT embeddings to bag-of-words
texts = [comment.split() for comment in df['Comment'].tolist()]
dictionary = Dictionary(texts)
corpus = [dictionary.doc2bow(text) for text in texts]

# Use KMeans centroids to find closest comments for each topic
def get_representative_comments(topic_idx, n_representative=20):
    indices = np.where(df['topic'] == topic_idx)[0]
    centroid = kmeans.cluster_centers_[topic_idx]
    distances = cosine_similarity([centroid], embeddings[indices].numpy())
    closest_indices = np.argsort(distances[0])[:n_representative]
    return [df.iloc[indices[i]]['Comment'] for i in closest_indices]

def get_most_frequent_words(comments, top_n=10):
    # Tokenize words, convert to lower case, and filter out stop words and punctuation
    words = [word for comment in comments
             for word in word_tokenize(comment.lower())
             if word not in stop_words and word not in string.punctuation]

    # Find the most common words
    most_common = [word for word, freq in Counter(words).most_common(top_n)]
    return most_common

# Create lists of most frequent words representing each topic
topic_words = [get_most_frequent_words(get_representative_comments(i)) for i in range(num_topics)]
# Print topic_words in a readable format
print("Topic Words by Topic:")
for idx, words in enumerate(topic_words):
    print(f"Topic {idx + 1}: {', '.join(words)}")

cm = CoherenceModel(topics=topic_words, texts=texts, dictionary=dictionary, coherence='c_v')
#cm = CoherenceModel(topics=topic_words, dictionary=dictionary, coherence='u_mass')
coherence_score = cm.get_coherence()
print(f'Coherence Score: {coherence_score}')

Topic Words by Topic:
Topic 1: new, model, also, sale, --, offer, monitor, '', brand, controller
Topic 2: --, would, n't, like, 'm, ``, '', 's, know, mail
Topic 3: n't, 's, ``, '', chronic, lambda, l, think, two, want
Topic 4: --, '', 's, dk, ``, would, n't, use, ..., like
Topic 5: --, '', ``, 's, new, boston, people, pittsburgh, chicago, los
Topic 6: '', n't, 's, color, like, would, bus, key, 'd, dos
Topic 7: file, ..., object, program, window, would, //, using, mywindow, mywinobj
Topic 8: please, thanks, anyone, would, 's, --, times, know, could, first
Topic 9: --, '', one, 's, ``, n't, includes, 'm, officer, could
Topic 10: 's, --, '', ``, n't, one, would, god, jesus, think
Coherence Score: 0.33834595136159534
